In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython

import copy
import itertools
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import msprime
import numpy as np
import pandas as pd
import random
import toytree

from collections import Counter

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

## ClaDS return type

    list("tree"=obj,"times"=times,"nblineages"=nblineages,"rates"=rates,
    "lamb"=lamb,"mu"=mu,"maxRate"=tooHigh,"root_length"=root_length))

In [2]:
%%R -o res
source("../sim_clads.R")
res = sim_ClaDS(lambda_0=0.7, mu_0=0.5, taxa_stop=5, condition="taxa", prune_extinct=T)

[1] 0.185024
[1] 0.4912975
[1] 0.1513556
[1] 0.1748231
[1] 0.7822899
[1] 0.3541062
[1] 0.2789691
[1] 0.3360868
[1] 0.1267184
[1] 0.4728079
[1] 0.1103108


In [3]:
res

R object with classes: ('list',) mapped to:
[Lis..., Flo..., Flo..., Flo..., Flo..., Flo..., Boo..., Flo...]
  tree: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7fefdd984d48> [RTYPES.VECSXP]
  times: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefdd984e88> [RTYPES.REALSXP]
  nblineages: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefdd984e08> [RTYPES.REALSXP]
  rates: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefdd984e88> [RTYPES.REALSXP]
  lamb: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefdd984f88> [RTYPES.REALSXP]
  mu: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefd0674188> [RTYPES.REALSXP]
  maxRate: <class 'rpy2.rinterface.BoolSexpVector'>
  <rpy2.rinterface.BoolSexpVector object at 0x7fefdd984f88> [RTYPES.LGLSXP]
  root_length: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x7fefd0674208> [RTYPES.REALSXP]